1. Приложен ноутбук, в котором реализованы функции для генирации из большого
датасета меньшая его копия. Вам нужно перенести функции из этого ноутбука в
класс датасет и сделать следующее:
a. Сгенерировать меньший датасет из 8-10 классов движения
b. Обучить уже существующую модель (предварительно проанализировав
какие параметры модели нужно изменить)
c. Изменить модель: посмотреть зависимость от количества LSTM модулей
в нашей модели
d. Сгенерировать другой датасет с меньшим количеством “кадров” в серии
и сравнить улучшилось или ухудшилось качество предсказания.
Провести несколько таких итераций, дать свою оценку уменьшению и
увеличению кадров, назвать оптимальное, на ваш взгляд, их количество.
Желательно сделать так, чтобы длина последовательности
передавалась как атрибут класса

In [2]:
import torch
from torch import nn
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from torch import optim
import time
import math

In [4]:
data_path = "D:/nturgb+d_skeletons/"
broken_files_path = "D:/NTU_RGBD_samples_with_missing_skeletons.txt"

In [5]:
training_classes = sorted([8, 10, 22, 23, 27, 21, 5, 29]) 
LABELS = {x: training_classes[x] for x in range(len(training_classes))}
training_cameras = [1, 2, 3] 
num_joint = 25
max_frame = 300

In [17]:
class Dataset(Dataset):
    def __init__(self, data_path, broken_files_path=None, training_classes=None,
                 num_joint = 25, max_frame = 300, transform=None):
        
        
        def read_data(data_path, broken_files_path):
            labels = []
            files = []
            action_classes = {}
            counter = 0
            files_counter = {}
            with open(broken_files_path, 'r') as f:
                broken_files = f.read().split("\n")

            raw_files = os.listdir(data_path)
            num_frames = 0

            for filename in raw_files:
                if filename not in broken_files:
                    action_class = int(filename[filename.find('A') + 1:filename.find('A') + 4])
                    subject_id = int(filename[filename.find('P') + 1:filename.find('P') + 4])
                    camera_id = int(filename[filename.find('C') + 1:filename.find('C') + 4])
                    if action_class in training_classes and camera_id in training_cameras: 
                        if action_class in action_classes:
                            if files_counter[action_class] < 120:
                                files.append([filename,action_classes[action_class]])
                                files_counter[action_class] = files_counter[action_class] + 1
                        else:
                            action_classes.update({action_class : counter})
                            files_counter.update({action_class : 1})
                            counter+=1
                            files.append([filename,action_classes[action_class]])
            print("action classes: ", action_classes)
            print("action files: ", files_counter)

            return files, action_classes
        
        def read_skeleton_filter(file):
            with open(file, 'r') as f:
                skeleton_sequence = {}
                skeleton_sequence['numFrame'] = int(f.readline())
                skeleton_sequence['frameInfo'] = []
                for t in range(skeleton_sequence['numFrame']):
                    frame_info = {}
                    frame_info['numBody'] = int(f.readline())
                    frame_info['bodyInfo'] = []

                    for m in range(frame_info['numBody']):
                        body_info = {}
                        body_info_key = [
                            'bodyID', 'clipedEdges', 'handLeftConfidence',
                            'handLeftState', 'handRightConfidence', 'handRightState',
                            'isResticted', 'leanX', 'leanY', 'trackingState'
                        ]
                        body_info = {
                            k: float(v)
                            for k, v in zip(body_info_key, f.readline().split())
                        }
                        body_info['numJoint'] = int(f.readline())
                        body_info['jointInfo'] = []
                        for v in range(body_info['numJoint']):
                            joint_info_key = [
                                'x', 'y', 'z', 'depthX', 'depthY', 'colorX', 'colorY',
                                'orientationW', 'orientationX', 'orientationY',
                                'orientationZ', 'trackingState'
                            ]
                            joint_info = {
                                k: float(v)
                                for k, v in zip(joint_info_key, f.readline().split())
                            }
                            body_info['jointInfo'].append(joint_info)
                        frame_info['bodyInfo'].append(body_info)
                    skeleton_sequence['frameInfo'].append(frame_info)

            return skeleton_sequence
        
        def read_xyz(file, max_body=1, num_joint=25):
            seq_info = read_skeleton_filter(file)
            data = np.zeros((max_body, seq_info['numFrame'], num_joint, 3))
            for n, f in enumerate(seq_info['frameInfo']):
                for m, b in enumerate(f['bodyInfo']):
                    for j, v in enumerate(b['jointInfo']):
                        if m < max_body and j < num_joint:
                            data[m, n, j, :] = [v['x'], v['y'], v['z']]

                        else:
                            pass

            return data
        
        
        def create_coords_blocks(test_file, chonk_len = 45):   
            frame_counter = 0
            new_labels = []
            new_frames = []
            blocks = []

            test_frames = read_xyz(data_path + test_file[0])[0]
            label = test_file[1]
            slice_len = chonk_len * int(len(test_frames)/chonk_len)


            for index in range(len(test_frames[:slice_len])):
                frame_counter += 1
                new_frames.append(test_frames[index].flatten())
                if frame_counter == chonk_len:
                    frame_counter = 0
                    blocks.append(np.array(new_frames))
                    new_labels = new_labels + [label]
                    new_frames = []


            return blocks, new_labels
        
        
        working_files_with_labels, action_classes = read_data(data_path, broken_files_path)
        
        data = []
        labels = []
        ##########################################################################
        numbers = {x: 0 for x in range(len(action_classes))}  #####
        ##################################################################
        for file in working_files_with_labels:
            frames_blocks, label = create_coords_blocks(file)
            if label != [] and numbers[label[0]] <= 150:
                numbers[label[0]] = numbers[label[0]] + len(label)
                data = data + frames_blocks
                labels = labels + label
        data_np = np.asarray(data)
        labels_np = np.asarray(labels)

        data_sq = data_np.reshape(len(data_np), -1)
        data = pd.DataFrame(data_sq)
        labels = pd.DataFrame(labels_np)
        data['labels'] = labels
        

        self.data = data
        self.labels = data['labels'].astype('float32')
        self.transform = transform
        
           
    def __len__(self):
         return len(self.data)
        
        
    def __getitem__(self, idx):
        item = np.asarray(self.data.iloc[idx,:-1]).reshape(45,75)
        label = self.labels[idx]
        if self.transform != None:
            item = transform(item)
        return (item, label)

In [18]:
dataset = Dataset(data_path=data_path, broken_files_path=broken_files_path, 
                           training_classes=training_classes,num_joint = 25, 
                           max_frame = 300, transform=None)

action classes:  {5: 0, 10: 1, 21: 2, 23: 3, 27: 4, 29: 5, 8: 6, 22: 7}
action files:  {5: 120, 10: 120, 21: 120, 23: 120, 27: 120, 29: 120, 8: 120, 22: 120}


In [19]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(0.70*len(dataset)),int(0.30*len(dataset))])
train_loader = DataLoader(train_dataset, batch_size = 16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size = 4, shuffle=True)

ValueError: Sum of input lengths does not equal the length of the input dataset!

И вот тут все встало( Вроде ничего такого не сделано, просто функции из Data preparation оформлены в класс. Поиск по ошибке ничего не выдает кроме нескольких репо на гитхабе с такой же ошибкой. Возможно архив с датасетом в диск со свободными 100 ГБ распаковывается не полностью, выдает ошибку что недостаточно места. Распакованый он занимает 5,72 ГБ. Если дело в этом скажите пожалуйста как это можно поправить, памяти как будто достаточно но выходит ошибка и видимо не все файлы распаковываются. 